In [118]:
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D

Given a system of two coupled Schrödinger equations with the following non-linearity:
\begin{align}
iu_t + u_{xx} + \left(|u|^2 + \alpha|u|^4 + |v|^2\right)u & = 0; \\
iv_t + v_{xx} + \left(|u|^2 + |v|^2 + \alpha|v|^4\right)v & = 0,
\end{align} 
where $\alpha$ is a parameter. The boundary conditions are given by:
\begin{align}
u(-L,t)=u(L,t), \quad u_x(-L,t)=u_x(L,t); \\
v(-L,t)=v(L,t), \quad v_x(-L,t)=v_x(L,t).
\end{align}
Further, the initial conditions are given by:
\begin{align}
u(x,0) &= \sum_{j=1,2} \sech \left[(x-x_j)\right]\exp(ic_j(x-x_j); \\
v(x,0) &= \sum_{j=1,2} \sech \left[(x-x_j)\right]\exp(ic_j(x-x_j),
\end{align}
where $|c_j|<2$ are the velocities of the colliding solitions.

We want to explore the soliton collisions for different values of the parameter $\alpha$, the velocity $c_j$, and different values of the initial inter-soliton distance $|x_2 x_1|$. In particular, examining the effect of positive and negative values of $\alpha$ as well as small and large velocities $c_j$ and initial inter-soliton distances.

Defining the spatial step and spatial domain:

In [119]:
N = 100
L = 10
h = L / N
x = np.arange(-L, L, h)

Defining the time step and time domain:

In [120]:
t_max = 10
tau = (h**2)/8
t = np.arange(0, t_max + tau, tau)

Defining two zero matrices that will eventually store all the $u$ and $v$ values:

In [121]:
m, n = len(x), len(t)
u = np.zeros((m, n), dtype=complex)
v = np.zeros((m, n), dtype=complex)

Defining the initial conditions $u(x,0)$ and $v(x,0)$:

In [122]:
def sech_f(x):
    return 1 / np.cosh(x)

def u0_f(x, x1, x2, c1, c2):
    term_1 = sech_f(x - x1) * np.exp(1j * c1 * (x - x1))
    term_2 = sech_f(x - x2) * np.exp(1j * c2 * (x - x2))
    return term_1 + term_2

def v0_f(x, x1, x2, c1, c2):
    term_1 = sech_f(x - x1) * np.exp(1j * c1 * (x - x1))
    term_2 = sech_f(x - x2) * np.exp(1j * c2 * (x - x2))
    return term_1 + term_2

Calculating the initial conditions at $u(x,0)$ and $v(x,0)$:

In [123]:
x1, x2, c1, c2 = -5, 5, 1, 1
alpha = 0.5
u[:, 0] = u0_f(x, x1, x2, c1, c2)
v[:, 0] = v0_f(x, x1, x2, c1, c2)

#### Finite-difference method

Using the finite-difference approximations:
\begin{align}
u_{j,k+1} &= u_{j,k-1} + 2\tau i \left( \frac{u_{j-1,k} - 2u_{j,k} + u_{j+1,k}}{h^2}\right) + 2\tau i \left( |u_{j,k}|^2 + \alpha|u_{j,k}|^4 + |v_{j,k}|^2 \right) u_{j,k}; \\
v_{j,k+1} &= v_{j,k-1} + 2\tau i \left( \frac{v_{j-1,k} - 2v_{j,k} + v_{j+1,k}}{h^2}\right) + 2\tau i \left( |u_{j,k}|^2 + |v_{j,k}|^2 + \alpha|v_{j,k}|^4 \right) v_{j,k},
\end{align}
where the boundary points are treated using:
\begin{align}
u_{0,k}&=u_{N,k},\quad& v_{0,k}&=v_{N,k}; \\
u_{-1,k}&=u_{N-1,k},\quad& v_{-1,k}&=v_{N-1,k}; \\
u_{N+1,k}&=u_{1,k},\quad& v_{N+1,k}&=v_{1,k}.
\end{align}

These expressions can be simplified to:
\begin{align}
\vec{u}^{\,(k+1)} &= \vec{u}^{\,(k-1)} + D_1^{[j,k]}\vec{u}^{\,(k)}; \\
\vec{v}^{\,(k+1)} &= \vec{v}^{\,(k-1)} + D_2^{[j,k]}\vec{v}^{\,(k)},
\end{align}
where $D_1^{[j,k]}, D_2^{[j,k]}$ are the tridiagonal matrices that contain the coefficients for the finite difference approximations:
\begin{equation}
D_1^{[j,k]}=
\begin{pmatrix}
-\frac{4\tau i}{h^2} + 2\tau i\left(|u_{0,k}|^2+\alpha|u_{0,k}|^4+|v_{0,k}|^2\right) & \frac{2\tau i}{h^2} & 0 & \cdots & \frac{2\tau i}{h^2} \\
\frac{2\tau i}{h^2} & -\frac{4\tau i}{h^2} + 2\tau i\left(|u_{0,k}|^2+\alpha|u_{0,k}|^4+|v_{0,k}|^2\right) & \frac{2\tau i}{h^2} &  \cdots & 0 \\
\vdots & & & & \vdots \\
\frac{2\tau i}{h^2} & 0 & \cdots & \frac{2\tau i}{h^2} & -\frac{4\tau i}{h^2} + 2\tau i\left(|u_{0,k}|^2+\alpha|u_{0,k}|^4+|v_{0,k}|^2\right)
\end{pmatrix};
\end{equation}
\begin{equation}
D_2^{[j,k]}=
\begin{pmatrix}
-\frac{4\tau i}{h^2} + 2\tau i\left(|u_{0,k}|^2+|v_{0,k}|^2+\alpha|v_{0,k}|^4\right) & \frac{2\tau i}{h^2} & 0 & \cdots & \frac{2\tau i}{h^2} \\
\frac{2\tau i}{h^2} & -\frac{4\tau i}{h^2} + 2\tau i\left(|u_{0,k}|^2+|v_{0,k}|^2+\alpha|v_{0,k}|^4\right) & \frac{2\tau i}{h^2} &  \cdots & 0 \\
\vdots & & & & \vdots \\
\frac{2\tau i}{h^2} & 0 & \cdots & \frac{2\tau i}{h^2} & -\frac{4\tau i}{h^2} + 2\tau i\left(|u_{0,k}|^2+|v_{0,k}|^2+\alpha|v_{0,k}|^4\right)
\end{pmatrix}.
\end{equation}

In [124]:
def layer_matrix_u_f(u_layer, v_layer):
    upper_diagonal = np.diag([2*tau*1j/(h**2)]* (m - 1), 1)
    lower_diagonal = np.diag([2*tau*1j/(h**2)]* (m - 1), -1)
    main_diagonal = np.diag([-4*tau*1j/(h**2)+2*tau*1j*(np.abs(u_layer[i])**2 + alpha * np.abs(u_layer[i])**4 + np.abs(v_layer[i])**2) for i in range(len(u_layer))])
    
    val_mat = upper_diagonal + lower_diagonal + main_diagonal
    val_mat[(m - 1, 0)] = (2 * 1j * tau) / (h ** 2)
    val_mat[(0, m - 1)] = (2 * 1j * tau) / (h ** 2)
    
    return val_mat

def layer_matrix_v_f(u_layer, v_layer):
    upper_diagonal = np.diag([2*tau*1j/(h**2)]* (m - 1), 1)
    lower_diagonal = np.diag([2*tau*1j/(h**2)]* (m - 1), -1)
    main_diagonal = np.diag([-4*tau*1j/(h**2)+2*tau*1j*(np.abs(u_layer[i])**2 + np.abs(v_layer[i])**2 + alpha * np.abs(v_layer[i])**4) for i in range(len(v_layer))])
    
    val_mat = upper_diagonal + lower_diagonal + main_diagonal
    val_mat[(m - 1, 0)] = (2 * 1j * tau) / (h ** 2)
    val_mat[(0, m - 1)] = (2 * 1j * tau) / (h ** 2)
    
    return val_mat

The finite-difference approximations for the first time layer are:
\begin{align}
u_{j,k+1} &= u_{j,k} + \tau i \left( \frac{u_{j-1,k} - 2u_{j,k} + u_{j+1,k}}{h^2}\right) + \tau i \left( |u_{j,k}|^2 + \alpha|u_{j,k}|^4 + |v_{j,k}|^2 \right) u_{j,k}; \\
v_{j,k+1} &= v_{j,k} + \tau i \left( \frac{v_{j-1,k} - 2v_{j,k} + v_{j+1,k}}{h^2}\right) + \tau i \left( |u_{j,k}|^2 + |v_{j,k}|^2 + \alpha|v_{j,k}|^4 \right) v_{j,k},
\end{align}
where the boundary points are treated using:
\begin{align}
u_{0,k}&=u_{N,k},\quad& v_{0,k}&=v_{N,k}; \\
u_{-1,k}&=u_{N-1,k},\quad& v_{-1,k}&=v_{N-1,k}; \\
u_{N+1,k}&=u_{1,k},\quad& v_{N+1,k}&=v_{1,k}.
\end{align}

These expressions can be simplified to:
\begin{align}
\vec{u}^{\,(k+1)} &= \tilde{D}_1^{[j,k]}\vec{u}^{\,(k)}; \\
\vec{v}^{\,(k+1)} &= \tilde{D}_2^{[j,k]}\vec{v}^{\,(k)}, 
\end{align}
where $\tilde{D}_1^{[j,k]},\tilde{D}_2^{[j,k]}$ are the tridiagonal matrices that contain the coefficients for the finite difference approximations:
\begin{equation}
\tilde{D}_1^{[j,k]} =
\begin{pmatrix}
1 - \frac{2\tau i}{h^2} + \tau i \left(|u_{0,k}|^2 + \alpha|u_{0,k}|^4 + |v_{0,k}|^2 \right) & \frac{\tau i}{h^2} & 0 & \cdots & \frac{\tau i}{h^2} \\
\frac{\tau i}{h^2} & 1 - \frac{2\tau i}{h^2} + \tau i \left(|u_{1,k}|^2 + \alpha|u_{1,k}|^4 + |v_{1,k}|^2 \right) & \frac{\tau i}{h^2} & \cdots & 0 \\
\vdots & & & & \vdots \\
\frac{\tau i}{h^2} & \cdots & 0 & \frac{\tau i}{h^2} & 1 - \frac{2\tau i}{h^2} + \tau i \left(|u_{N,k}|^2 + \alpha|u_{N,k}|^4 + |v_{N,k}|^2 \right)
\end{pmatrix};
\end{equation}
\begin{equation}
\tilde{D}_2^{[j,k]} =
\begin{pmatrix}
1 - \frac{2\tau i}{h^2} + \tau i \left(|u_{0,k}|^2 + |v_{0,k}|^2 + \alpha|v_{0,k}|^4 \right) & \frac{\tau i}{h^2} & 0 & \cdots & \frac{\tau i}{h^2} \\
\frac{\tau i}{h^2} & 1 - \frac{2\tau i}{h^2} + \tau i \left(|u_{1,k}|^2 + |v_{1,k}|^2 + \alpha|v_{1,k}|^4 \right) & \frac{\tau i}{h^2} & \cdots & 0 \\
\vdots & & & & \vdots \\
\frac{\tau i}{h^2} & \cdots & 0 & \frac{\tau i}{h^2} & 1 - \frac{2\tau i}{h^2} + \tau i \left(|u_{N,k}|^2 + |v_{N,k}|^2 + \alpha|v_{N,k}|^4 \right)
\end{pmatrix}.
\end{equation}

In [125]:
def first_layer_matrix_u_f(u_value, v_value):
    upper_diagonal = np.diag([tau*1j/(h**2)] * (m - 1), 1)
    lower_diagonal = np.diag([tau*1j/(h**2)] * (m - 1), -1)
    main_diagonal = np.diag([1-(2*tau*1j/(h**2) + tau*1j*(np.abs(u_value[i])**2 + alpha*np.abs(u_value[i])**4 + np.abs(v_value[i])**2)) for i in range(len(u_value))])
    
    val_mat = upper_diagonal + lower_diagonal + main_diagonal
    val_mat[(m - 1, 0)] = (1j * tau) / (h ** 2)
    val_mat[(0, m - 1)] = (1j * tau) / (h ** 2)
    
    return val_mat

def first_layer_matrix_v_f(u_value, v_value):
    upper_diagonal = np.diag([tau*1j/(h**2)] * (m - 1), 1)
    lower_diagonal = np.diag([tau*1j/(h**2)] * (m - 1), -1)
    main_diagonal = np.diag([1-(2*tau*1j/(h**2) + tau*1j*(np.abs(u_value[i])**2 + np.abs(v_value[i])**2 + alpha*np.abs(v_value[i])**4)) for i in range(len(u_value))])
    
    val_mat = upper_diagonal + lower_diagonal + main_diagonal
    val_mat[(m - 1, 0)] = (1j * tau) / (h ** 2)
    val_mat[(0, m - 1)] = (1j * tau) / (h ** 2)
    
    return val_mat

We can now solve for all values of $u$ and $v$ for each time at $k=0,1,\ldots$:

In [126]:
u[:, 1] = np.matmul(first_layer_matrix_u_f(u[:, 0], v[:, 0]), u[:, 0])
v[:, 1] = np.matmul(first_layer_matrix_u_f(u[:, 0], v[:, 0]), v[:, 0])

In [127]:
for k in range(1, n-1):
    u[:, k + 1] = u[:, k - 1] + np.matmul(layer_matrix_u_f(u[:, k], v[:, k]), u[:, k])
    v[:, k + 1] = v[:, k - 1] + np.matmul(layer_matrix_v_f(u[:, k], v[:, k]), v[:, k])